In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
import pickle


In [2]:
data=pd.read_csv("Churn_Modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
# Preprocessing and model training code would go here

data=data.drop(['RowNumber','CustomerId','Surname'],axis=1)

In [4]:
## Encode categorical variables
label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])

In [5]:
## One-hot encode 'Geography' column

onehot_encoder_geo=OneHotEncoder(handle_unknown='ignore')
geo_encoded = onehot_encoder_geo.fit_transform(data[['Geography']]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [6]:
## combine one-hot encoded columns with original dataframe
data = pd.concat([data.drop('Geography',axis=1), geo_encoded_df], axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [7]:
## Split the data into features and target variable
x=data.drop('EstimatedSalary',axis=1)
y=data['EstimatedSalary']

In [8]:
## spilt the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [14]:
## scale these features
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [15]:
## Save the encoders and scaler for future use
with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender, file)


with open('onehot_encoder_geo.pkl', 'wb') as file:
    pickle.dump(onehot_encoder_geo, file)

with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)



ANN Regression Problem statement

In [16]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


In [17]:
## Build the ANN model
model= Sequential([
    Dense(64, activation='relu', input_shape=(x_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1)  # Output layer for regression
])


## compile the model
model.compile(optimizer='adam', loss='mean_absolute_error', metrics=['mae'])

model.summary()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [18]:
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

#set up TensorBoard callback
log_dir = "regressionlogs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [19]:
## Set up early stopping

early_stoppping_callback = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)


In [20]:
## train the model
history = model.fit(
    x_train, y_train,
    validation_data=(x_test, y_test),
    epochs=100,
    callbacks=[early_stoppping_callback, tensorboard_callback]
)

Epoch 1/100


250/250 [==============================] - 2s 5ms/step - loss: 100379.5938 - mae: 100379.5938 - val_loss: 98521.6484 - val_mae: 98521.6484
Epoch 2/100
250/250 [==============================] - 1s 3ms/step - loss: 99638.7969 - mae: 99638.7969 - val_loss: 97028.0391 - val_mae: 97028.0391
Epoch 3/100
250/250 [==============================] - 1s 3ms/step - loss: 97040.5391 - mae: 97040.5391 - val_loss: 93233.9219 - val_mae: 93233.9219
Epoch 4/100
250/250 [==============================] - 1s 5ms/step - loss: 91922.1562 - mae: 91922.1562 - val_loss: 86878.6172 - val_mae: 86878.6172
Epoch 5/100
250/250 [==============================] - 1s 5ms/step - loss: 84436.0234 - mae: 84436.0234 - val_loss: 78695.3828 - val_mae: 78695.3828
Epoch 6/100
250/250 [==============================] - 1s 4ms/step - loss: 75675.3359 - mae: 75675.3359 - val_loss: 70048.8203 - val_mae: 70048.8203
Epoch 7/100
250/250 [==============================] - 1s 4ms/step - loss: 67141.1094 - mae: 67141.109

In [22]:
%load_ext tensorboard

In [24]:
%tensorboard --logdir regressionlogs/fit

Reusing TensorBoard on port 6007 (pid 24372), started 0:00:41 ago. (Use '!kill 24372' to kill it.)

In [25]:
## evaluate the model

test_loss, test_mae = model.evaluate(x_test, y_test)
print(f'Test MAE: {test_mae}')


63/63 [==============================] - 0s 5ms/step - loss: 50311.8594 - mae: 50311.8594
Test MAE: 50311.859375


In [27]:
model.save('regression_model.h5')
